In [1]:
from importlib import reload
#reload(Utilities)
#reload(clm)
# NOTE: To reload a class imported as, e.g., 
# from module import class
# One must call:
#   1. import module
#   2. reload module
#   3. from module import class

import sys, os
import re

import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype
from scipy import stats
import datetime
import time
from natsort import natsorted, ns
from packaging import version

import itertools

import pyodbc
#---------------------------------------------------------------------
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib.ticker as ticker
from matplotlib import dates
#---------------------------------------------------------------------
sys.path.insert(0, os.path.realpath('..'))
import Utilities_config
#-----
import CommonLearningMethods as clm
#-----
from MeterPremise import MeterPremise
#-----
from AMI_SQL import AMI_SQL
from AMINonVee_SQL import AMINonVee_SQL
from AMIEndEvents_SQL import AMIEndEvents_SQL
from AMIUsgInst_SQL import AMIUsgInst_SQL
from DOVSOutages_SQL import DOVSOutages_SQL
#-----
from GenAn import GenAn
from AMINonVee import AMINonVee
from AMIEndEvents import AMIEndEvents
from AMIUsgInst import AMIUsgInst
from DOVSOutages import DOVSOutages
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_sql_aids_dir())
import Utilities_sql
import TableInfos
from TableInfos import TableInfo
from SQLElement import SQLElement
from SQLElementsCollection import SQLElementsCollection
from SQLSelect import SQLSelectElement, SQLSelect
from SQLFrom import SQLFrom
from SQLWhere import SQLWhereElement, SQLWhere
from SQLJoin import SQLJoin, SQLJoinCollection
from SQLGroupBy import SQLGroupByElement, SQLGroupBy
from SQLHaving import SQLHaving
from SQLOrderBy import SQLOrderByElement, SQLOrderBy
from SQLQuery import SQLQuery
from SQLQueryGeneric import SQLQueryGeneric
#---------------------------------------------------------------------
#sys.path.insert(0, os.path.join(os.path.realpath('..'), 'Utilities'))
sys.path.insert(0, Utilities_config.get_utilities_dir())
import Utilities
import Utilities_df
from Utilities_df import DFConstructType
import Utilities_dt
import Plot_Box_sns
import GrubbsTest

In [2]:
conn_aws = Utilities.get_athena_prod_aws_connection()

In [3]:
save_pkl=True
save_dir_base = r'C:\Users\s346557\Documents\LocalData\xfmr_mappings'
snapshots_subdir = 'snapshots'

include_PNs = True
include_SNs = True

addtnl_grpby_cols = ['state_cd', 'circuit_nb', 'station_nb']
#--------------------------------------------------
assert(include_PNs+include_SNs>0)
#-----
grpby_cols = ['trsf_pole_nb']
if addtnl_grpby_cols is not None:
    grpby_cols.extend(addtnl_grpby_cols)
#--------------------------------------------------
if save_pkl and not os.path.exists(os.path.join(save_dir_base, snapshots_subdir)):
    os.makedirs(os.path.join(save_dir_base, snapshots_subdir))
#--------------------------------------------------

In [4]:
#-------------------------
# sql = """
# SELECT {0}{1}
# FROM default.meter_premise
# WHERE state_cd='OH'
# GROUP BY {0}
# """.format(', '.join([str(x) for x in grpby_cols]), '{}')

sql = """
SELECT {0}{1}
FROM default.meter_premise
GROUP BY {0}
""".format(', '.join([str(x) for x in grpby_cols]), '{}')
#-----
array_aggs = ''
if include_PNs:
    array_aggs += ", array_join(array_agg(prem_nb),', ') AS PNs"
if include_SNs:
    array_aggs += ", array_join(array_agg(mfr_devc_ser_nbr),', ') AS SNs"
#-----
sql = sql.format(array_aggs)
#-------------------------

In [5]:
print(sql)


SELECT trsf_pole_nb, state_cd, circuit_nb, station_nb, array_join(array_agg(prem_nb),', ') AS PNs, array_join(array_agg(mfr_devc_ser_nbr),', ') AS SNs
FROM default.meter_premise
GROUP BY trsf_pole_nb, state_cd, circuit_nb, station_nb



In [6]:
start=time.time()
df = pd.read_sql_query(sql, conn_aws)
print(time.time()-start)
#-------------------------
# The elements are returns as strings, not lists.  So, they must be converted into lists
#   using the str.split functionality.
# The purpose of the apply lambda function is to remove leading/trailing white space
#   from each element within a given row's list.
#   If the separator fed into split is exactly that used in the SQL query, then this step is
#     unnecessary.  However, if the two differ, e.g., ', ' used in SQL query but ',' fed to split
#     (notice the extra whitespace in the former), then it will be needed!
for col in [x for x in df.columns if x not in grpby_cols]:
    df[col] = df[col].str.split(', ').apply(lambda x: sorted([i.strip() for i in x]))

95.35379648208618


In [7]:
# # Below from other notebook (no longer exists), and will be useful if 
# #   one wants to add where statements
# include_PNs = True
# include_SNs = True

# addtnl_grpby_cols = ['state_cd', 'circuit_nb', 'station_nb']
# #--------------------------------------------------
# assert(include_PNs+include_SNs>0)
# #-----
# grpby_cols = ['trsf_pole_nb']
# if addtnl_grpby_cols is not None:
#     grpby_cols.extend(addtnl_grpby_cols)
# #--------------------------------------------------
# # sql = """
# # SELECT {0}{1}
# # FROM default.meter_premise
# # WHERE state_cd='OH'
# # GROUP BY {0}
# # """.format(', '.join([str(x) for x in grpby_cols]), '{}')

# sql = """
# SELECT {0}{1}
# FROM default.meter_premise{2}
# GROUP BY {0}
# """.format(', '.join([str(x) for x in grpby_cols]), '{0}', '{1}')
# #-----
# array_aggs = ''
# if include_PNs:
#     array_aggs += ", array_join(array_agg(prem_nb),', ') AS PNs"
# if include_SNs:
#     array_aggs += ", array_join(array_agg(mfr_devc_ser_nbr),', ') AS SNs"
# #-------------------------
# # where_kwargs = dict(premise_nbs=PNs[:10])
# where_kwargs = dict(premise_nbs=mp_df_PNs_0['prem_nb'].unique().tolist(), premise_nb_col='prem_nb')

# sql_where = SQLWhere()
# sql_where = AMI_SQL.add_ami_where_statements(sql_where, **where_kwargs)
# #-----
# sql = sql.format(array_aggs ,'\n'+sql_where.get_statement_string())
# #-------------------------
# start=time.time()
# df = pd.read_sql_query(sql, conn_aws)
# print(time.time()-start)
# #-------------------------
# # The elements are returns as strings, not lists.  So, they must be converted into lists
# #   using the str.split functionality.
# # The purpose of the apply lambda function is to remove leading/trailing white space
# #   from each element within a given row's list.
# #   If the separator fed into split is exactly that used in the SQL query, then this step is
# #     unnecessary.  However, if the two differ, e.g., ', ' used in SQL query but ',' fed to split
# #     (notice the extra whitespace in the former), then it will be needed!
# for col in [x for x in df.columns if x not in grpby_cols]:
#     df[col] = df[col].str.split(', ').apply(lambda x: sorted([i.strip() for i in x]))

In [8]:
df

,trsf_pole_nb,state_cd,circuit_nb,station_nb,PNs,SNs
0,40810500B39110,WV,12,1003,[063220134],[428204339]
1,40810523C25863,WV,37,1005,[063920130],[428781539]
2,40810719B02019,WV,21,1026,[066460393],[428957240]
3,40810761A03182,WV,,,[060869283],[435503325]
4,40810809A03168,WV,01,4500,[064784792],[435789815]
...,...,...,...,...,...,...
1715898,37790200D00166,VA,03,8139,"[020806166, 028206166]","[790519819, 790519820]"
1715899,37800038C04021,VA,02,7115,[022929998],[790519828]
1715900,37800039C00056,VA,01,7115,[027983756],[790519917]
1715901,38810097B40417,WV,04,3473,"[024282090, 027382090]","[790522703, 790522705]"


In [9]:
if save_pkl:
    save_name = f'xfmr_mappings_snapshot_{datetime.date.today().strftime("%Y%m%d")}.pkl'
    df.to_pickle(os.path.join(save_dir_base, snapshots_subdir, save_name))

In [10]:
save_name

'xfmr_mappings_snapshot_20230410.pkl'

In [11]:
df_1 = pd.read_pickle(r'C:\Users\s346557\Documents\LocalData\xfmr_mappings\xfmr_mappings.pkl')
df_2 = pd.read_pickle(r'C:\Users\s346557\Documents\LocalData\xfmr_mappings\snapshots\{}'.format(save_name))

In [12]:
print(df_1.shape)
print(df_2.shape)

(1743950, 6)
(1715903, 6)


In [13]:
df_1

,trsf_pole_nb,state_cd,circuit_nb,station_nb,PNs,SNs
0,,IN,,,"[047415502, 046716342, 044738796, 046202547, 0...","[LW2105A608, LW2105A610, 996719241, 190580827,..."
1,,LA,12520,871,[963350773],[190804865]
2,,LA,5390,806,[962324479],[190829077]
3,,LA,7170,793,"[966892761, 969315536]","[536953330, 536953335]"
4,,MI,,,[042335672],"[LW2003A352, LW2003A355]"
...,...,...,...,...,...,...
1743945,`,OK,ZH-1,90,[952422190],[756412722]
1743946,unset,OK,1505,9,"[955813355, 955042523]","[563506443, 762389722]"
1743947,unset,OK,70852,708,"[954161743, 957261743]","[761046605, 761046606]"
1743948,unset,OK,85933,81859,[959444967],[764920988]


In [14]:
df_1['trsf_pole_nb'].nunique()

1712197

In [15]:
vcs_1 = df_1['trsf_pole_nb'].value_counts()
vcs_2 = df_2['trsf_pole_nb'].value_counts()

In [16]:
vcs_1[vcs_1>1]

PRIMARY           2805
9999999999999      242
NETWORK            121
UNSET               45
                    41
                  ... 
47139904568164       2
43636885354912       2
TI0049000053         2
43637005351318       2
TI0050000099         2
Name: trsf_pole_nb, Length: 27503, dtype: int64

In [17]:
vcs_1[vcs_1>10]

PRIMARY            2805
9999999999999       242
NETWORK             121
UNSET                45
                     41
999999999999999      33
TRANSMISSION         29
999999999999         25
999999               16
9999999              15
9999999999           13
9999                 13
99999                13
999999999            13
999                  12
9                    12
99999999999999       11
Name: trsf_pole_nb, dtype: int64

In [18]:
df_1[df_1['trsf_pole_nb']=='62880656820110']

,trsf_pole_nb,state_cd,circuit_nb,station_nb,PNs,SNs
1518152,62880656820110,LA,,,[965660468],[658439481]
1518153,62880656820110,LA,7170,793,"[969385786, 966653296, 968618330, 962959829, 9...","[658439481, 190808548, 533612094, 535479476, 1..."


In [19]:
df_1[df_1['trsf_pole_nb']=='1714063082']

,trsf_pole_nb,state_cd,circuit_nb,station_nb,PNs,SNs
60879,1714063082,OK,,,[957891099],[794310310]
60880,1714063082,OK,XK-3,705,"[952850983, 956430565, 958481304, 957891099, 9...","[770168774, 756412036, 764971126, 794310310, 7..."


In [20]:
df_1[df_1['trsf_pole_nb']=='999999999999999']

,trsf_pole_nb,state_cd,circuit_nb,station_nb,PNs,SNs
1555191,999999999999999,AR,,,"[965038927, 962889451, 967456764, 969363777, 9...","[534254701, 544773892, 533652936, 542790989, 5..."
1555192,999999999999999,AR,11160,892,[961744566],[435776112]
1555193,999999999999999,AR,16600,1163,[966316525],[656714226]
1555194,999999999999999,AR,R950,916,[969146228],[323671579]
1555195,999999999999999,LA,,,"[964615338, 967489887, 960504930, 967767755, 9...","[450784105, 534091872, 76784240, 428164526, 42..."
1555196,999999999999999,LA,17140,1006,[962119094],[534126086]
1555197,999999999999999,LA,1800,18,[965249608],[428995968]
1555198,999999999999999,LA,1D00,790,[965524475],[549944299]
1555199,999999999999999,LA,3730,807,[965606788],[688942224]
1555200,999999999999999,LA,5480,801,[961989307],[428640144]


In [21]:
vcs_1[(vcs_1>2) & (vcs_1<10)]

0                 9
NA                8
51596895087334    5
000000000000      5
123456            4
                 ..
38791138A17554    3
38820273D00006    3
44392385038815    3
44233455050173    3
44382255034809    3
Name: trsf_pole_nb, Length: 779, dtype: int64

In [22]:
df_1[df_1['trsf_pole_nb']=='50108684972657']

,trsf_pole_nb,state_cd,circuit_nb,station_nb,PNs,SNs
1256462,50108684972657,TX,,,[940283204],[200023111]
1256463,50108684972657,TX,1560,5085,[948062910],[109103677]
1256464,50108684972657,TX,1660,5039,[948350020],[148415951]


In [23]:
def consolidate_xfmr_mappings(
    xfmr_mapping_dfs,
    grpby_cols=['trsf_pole_nb', 'state_cd', 'circuit_nb', 'station_nb'], 
    list_cols=['PNs', 'SNs']
):
    r"""
    """
    #-------------------------
    # Make sure xfmr_mapping_dfs is a list of pd.DataFrame object
    assert(isinstance(xfmr_mapping_dfs, list))
    assert(Utilities.are_all_list_elements_of_type(xfmr_mapping_dfs, pd.DataFrame))
    #-------------------------
    # Make sure all DFs in xfmr_mapping_dfs have the same columns, and then concatenate
    cols = xfmr_mapping_dfs[0].columns.tolist()
    for df_i in xfmr_mapping_dfs:
        assert(len(set(cols).symmetric_difference(set(df_i.columns.tolist())))==0)
    return_df = pd.concat(xfmr_mapping_dfs)
    #-------------------------
    # It is also expected that the DFs contained in xfmr_mapping_dfs have only the columns
    # contained in grpby_cols and list_cols
    assert(len(set(cols).symmetric_difference(set(grpby_cols+list_cols)))==0)
    #-------------------------
    # Group by grpby_cols and combine the entries in the list cols
    return_df = return_df.groupby(grpby_cols, as_index=False)[list_cols].agg(lambda x: list(set(x.sum())))
    #-------------------------
    return return_df 

In [24]:
xfmr_mappings_df = consolidate_xfmr_mappings([df_1, df_2])

In [25]:
xfmr_mappings_df

,trsf_pole_nb,state_cd,circuit_nb,station_nb,PNs,SNs
0,,IN,,,"[046716342, 047415502, 044738796, 044315502, 0...","[LW2105A608, 190580827, LW2006A155, LW2105A609..."
1,,LA,12520,871,[963350773],[190804865]
2,,LA,5390,806,[962324479],[190829077]
3,,LA,7170,793,"[969315536, 966892761]","[536953335, 536953330]"
4,,MI,,,[042335672],"[LW2003A355, LW2003A352]"
...,...,...,...,...,...,...
1745232,`,OK,ZH-1,90,[952422190],[756412722]
1745233,unset,OK,1505,9,"[955813355, 955042523]","[563506443, 762389722]"
1745234,unset,OK,70852,708,"[954161743, 957261743]","[761046606, 761046605]"
1745235,unset,OK,85933,81859,[959444967],[764920988]


In [26]:
if save_pkl:
#     save_name = f'xfmr_mappings_{datetime.date.today().strftime("%Y%m%d")}.pkl'
    save_name = 'xfmr_mappings.pkl'
    xfmr_mappings_df.to_pickle(os.path.join(save_dir_base, save_name))

In [27]:
sorted(xfmr_mappings_df['trsf_pole_nb'].unique())[-20:]

['XX1320101',
 'XX26101',
 'XX618101',
 'XX727401',
 'XXX',
 'XXXXXX',
 'XXXXXXXXXX',
 'XXXXXXXXXXXXXXX',
 'Y0649407373084',
 'Y2118146886816',
 'Y2249066858108',
 'Y57710316242',
 'Y618299971',
 'Y7145866877329',
 'Y913361702',
 'Y914191088',
 'Y9444686697114',
 'Y9758657193173',
 '`',
 'unset']

In [28]:
xfmr_mappings_df.explode('PNs').explode('SNs')

,trsf_pole_nb,state_cd,circuit_nb,station_nb,PNs,SNs
0,,IN,,,046716342,LW2105A608
0,,IN,,,046716342,190580827
0,,IN,,,046716342,LW2006A155
0,,IN,,,046716342,LW2105A609
0,,IN,,,046716342,LW2105A611
...,...,...,...,...,...,...
1745236,unset,OK,XA-3,385,953483739,761789966
1745236,unset,OK,XA-3,385,959024131,761789984
1745236,unset,OK,XA-3,385,959024131,762444343
1745236,unset,OK,XA-3,385,959024131,761789985


In [29]:
xfmr_mappings_df['state_cd'].unique()

array(['IN', 'LA', 'MI', 'OH', 'OK', 'TX', 'VA', 'WV', 'AR', 'KY', 'TN'],
      dtype=object)

In [30]:
print(xfmr_mappings_df.shape[0])
print()
for state in xfmr_mappings_df['state_cd'].unique():
    print(state)
    print(xfmr_mappings_df[xfmr_mappings_df['state_cd']==state].shape[0])
    print()

1745237

IN
127612

LA
95839

MI
43081

OH
429332

OK
151167

TX
387811

VA
215897

WV
164090

AR
35321

KY
79958

TN
15129

